In [38]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.preprocessing import PowerTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.compose import TransformedTargetRegressor
from sklearn.cluster import KMeans

# import regressors
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor, StackingRegressor
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, SGDRegressor, PassiveAggressiveRegressor, Perceptron, RidgeClassifier, LogisticRegression
from sklearn.linear_model import Lasso, ElasticNet, Lars, BayesianRidge, ARDRegression, OrthogonalMatchingPursuit, HuberRegressor, TheilSenRegressor, RANSACRegressor
from sklearn.linear_model import LassoLars, LassoLarsIC
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

# pandas deactivate future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

SUBMIT = False
USE_ORIGINAL = True
SEED = 15
SAMPLE = 1

train = pd.read_csv('datasets/train.csv')
test = pd.read_csv('datasets/test.csv')
orig = pd.read_csv('datasets/cubic_zirconia.csv')

base_cols = train.columns

for i, df in enumerate([train, test, orig]):
    df.drop(['id'], axis=1, inplace=True)
    df.drop_duplicates(inplace=True)
    # df['dataset'] = i

# Define test set
if not SUBMIT:
    train, test = train_test_split(train, test_size=0.2, random_state=SEED) 

if USE_ORIGINAL:
    train = pd.concat([train, orig], axis=0)
    train.reset_index(inplace=True, drop=True)

# Sampling for faster training
if SAMPLE < 1:
    train = train.sample(frac=SAMPLE, random_state=SEED)

del orig

# set training data
X_train = train.copy()
y_train = X_train.pop('price')
X_test = test.copy()

if not SUBMIT:
    y_test = X_test.pop('price')
else:
    y_test = None

# transform categorical features
def transform_categorical(df):
    df['cut'] = df['cut'].map({'Fair': 0, 'Good': 1, 'Very Good': 2, 'Premium': 3, 'Ideal': 4})
    df['color'] = df['color'].map({'J': 0, 'I': 1, 'H': 2, 'G': 3, 'F': 4, 'E': 5, 'D': 6})
    df['clarity'] = df['clarity'].map({'I1': 0, 'SI2': 1, 'SI1': 2, 'VS2': 3, 'VS1': 4, 'VVS2': 5, 'VVS1': 6, 'IF': 7})
    return df

def remove_outliers(df):
    # Drop extreme values
    min = 2
    max = 20
    df = df[(df['x'] < max) & (df['y'] < max) & (df['z'] < max)]
    df = df[(df['x'] > min) & (df['y'] > min) & (df['z'] > min)]
    return df

def add_volume_ratio(df):
    df['volume_ratio1'] = (df['x'] * df['y']) / (df['z'] * df['z'])
    df['volume_ratio2'] = (df['x'] * df['z']) / (df['y'] * df['y'])
    df['volume_ratio3'] = (df['y'] * df['z']) / (df['x'] * df['x'])
    df['volume_ratio4'] = (df['x']) / (df['z'])
    df['volume_ratio5'] = (df['y']) / (df['z'])
    df['volume_ratio6'] = (df['x']) / (df['y'])
    df['volume_ratio7'] = (df['x'] + df['y']) / df['z']
    df['volume_ratio8'] = (df['x'] + df['z']) / df['y']
    df['volume_ratio9'] = (df['y'] + df['z']) / df['x']
    df['volume_ratio10'] = (df['x'] * df['y'] * df['z']) / (df['x'].mean() * df['y'].mean() * df['z'].mean())
    df['volume_ratio11'] = (df['x'] * df['y'] * df['z']) / (df['x'].max() * df['y'].max() * df['z'].max())
    # df['volume_ratio12'] = (df['x'] * df['y'] * df['z']) / (df['x'].min() * df['y'].min() * df['z'].min())
    df['volume_ratio13'] = (df['x'] * df['y'] * df['z']) / (df['x'].median() * df['y'].median() * df['z'].median())
    df['volume_ratio14'] = (df['x'] * df['y'] * df['z']) / (df['x'].std() * df['y'].std() * df['z'].std())
    return df

def feature_engineering(df):
    df["volume"] = df["x"] * df["y"] * df["z"]
    df["surface_area"] = 2 * (df["x"] * df["y"] + df["y"] * df["z"] + df["z"] * df["x"])
    df["aspect_ratio_xy"] = df["x"] / df["y"]
    df["aspect_ratio_yz"] = df["y"] / df["z"]
    df["aspect_ratio_zx"] = df["z"] / df["x"]
    df["diagonal_distance"] = np.sqrt(df["x"] ** 2 + df["y"] ** 2 + df["z"] ** 2)
    # df["relative_height"] = (df["z"] - df["z"].min()) / (df["z"].max() - df["z"].min())
    # df["relative_position"] = (df["x"] + df["y"] + df["z"]) / (df["x"] + df["y"] + df["z"]).sum()
    # df["volume_ratio"] = df["x"] * df["y"] * df["z"] / (df["x"].mean() * df["y"].mean() * df["z"].mean())
    # df["length_ratio"] = df["x"] / df["x"].mean()
    # df["width_ratio"] = df["y"] / df["y"].mean()
    # df["height_ratio"] = df["z"] / df["z"].mean()
    df["sphericity"] = 1.4641 * (6 * df["volume"])**(2/3) / df["surface_area"]
    df["compactness"] = df["volume"]**(1/3) / df["x"]
    df['density'] = df['carat'] / df['volume']
    df['table_percentage'] = (df['table'] / ((df['x'] + df['y']) / 2)) * 100
    df['depth_percentage'] = (df['depth'] / ((df['x'] + df['y']) / 2)) * 100
    df['symmetry'] = (abs(df['x'] - df['z']) + abs(df['y'] - df['z'])) / (df['x'] + df['y'] + df['z'])
    df['surface_area'] = 2 * ((df['x'] * df['y']) + (df['x'] * df['z']) + (df['y'] * df['z']))
    df['depth_to_table_ratio'] = df['depth'] / df['table']
    df['girdle_diameter'] = 100 * df['z'] / df['depth']
    df['girdle_thickness'] = 100 * df['z'] / df['table']
    df['girdle_ratio'] = df['girdle_diameter'] / df['girdle_thickness']
    return df

def target_transform(serie):
    serie = np.log1p(serie)
    return serie

def inverse_target_transform(serie):
    serie = np.expm1(serie)
    return serie

def set_categorical(df):
    df['cut'] = df['cut'].astype('category')
    df['color'] = df['color'].astype('category')
    df['clarity'] = df['clarity'].astype('category')
    return df

def add_girdle_parameters(df):
    df['girdle_diameter'] = 100 * df['z'] / df['depth']
    df['girdle_thickness'] = 100 * df['z'] / df['table']
    df['girdle_ratio'] = df['girdle_diameter'] / df['girdle_thickness']
    return df

def impute_x_y_z(df):
    df['is_imputed'] = df.isna().any(axis=1).astype(int)
    df['girdle_diameter'].fillna((df['x'] + df['y']) / 2, inplace=True)
    df['x'].fillna(2*df['girdle_diameter'] - df['y'], inplace=True)
    df['y'].fillna(2*df['girdle_diameter'] - df['x'], inplace=True)
    df['z'].fillna(df['girdle_diameter'] * df['depth'] / 100, inplace=True)
    df = add_girdle_parameters(df)
    return df

def set_nan(df):
    for col in ['x', 'y', 'z']:
        df[col].replace(0, np.nan, inplace=True)
    return df

def drop_girdle_parameters(df):
    df.drop(['girdle_diameter', 'girdle_thickness', 'girdle_ratio'], axis=1, inplace=True)
    return df

# Make data preparation pipeline
def data_prepation(X_train, X_test):
    
    for df in [X_train, X_test]:
        # df = set_nan(df)
        df = transform_categorical(df)
        # df = set_categorical(df)
        # df = add_girdle_parameters(df)
        # df = impute_x_y_z(df)
        # df = drop_girdle_parameters(df)
        
    
    # imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    # imputer = IterativeImputer(max_iter=10, random_state=0)
    # imputer = KNNImputer(n_neighbors=1, weights="uniform")
    # X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    # X_test = pd.DataFrame(imputer.transform(X_test), columns=X_test.columns, index=X_test.index)
    
    # selected_cols = base_cols
    # selected_cols = ['surface_area', 'clarity', 'color', 'cut', 'carat', 'depth_percentage', 'depth', 'compactness', 'depth_to_table_ratio']
    
    for df in [X_train, X_test]:
        df = add_volume_ratio(df)
        df = feature_engineering(df)
        # df.fillna(0, inplace=True)
        # df.drop([col for col in df.columns if col not in selected_cols], axis=1, inplace=True)
        
    # Scaling
    # scaler = PowerTransformer()
    # X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
    # X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
    
    # Clustering features
    # model = KMeans(n_clusters=20, random_state=42)
    # X_train['cluster'] = model.fit_predict(X_train)
    # X_test['cluster'] = model.predict(X_test)
        
    return X_train, X_test
            
data_prep_has_fit_method = False

if not data_prep_has_fit_method:
    X_train, X_test = data_prepation(X_train, X_test)
    X_train_prep, X_test_prep = X_train.copy(), X_test.copy()
else:
    X_train_prep, X_test_prep = data_prepation(X_train.copy(), X_test.copy())
    
# X_train_prep, X_test_prep = data_prepation(X_train.copy(), X_test.copy())
# pd.DataFrame(X_train_prep.isna().sum(), columns=['train']).join(pd.DataFrame(X_test_prep.isna().sum(), columns=['test']))
X_train.head()

,carat,cut,color,clarity,depth,table,x,y,z,volume_ratio1,...,sphericity,compactness,density,table_percentage,depth_percentage,symmetry,depth_to_table_ratio,girdle_diameter,girdle_thickness,girdle_ratio
0,1.38,4,5,1,61.7,56.0,7.11,7.17,4.40,2.633197,...,0.784066,0.854563,0.006152,784.313725,864.145658,0.293362,1.101786,7.131280,7.857143,0.907618
1,1.56,4,2,4,61.5,56.0,7.51,7.49,4.61,2.646793,...,0.783838,0.849117,0.006016,746.666667,820.000000,0.294748,1.098214,7.495935,8.232143,0.910569
2,0.36,4,6,1,60.9,56.0,4.59,4.61,2.81,2.679791,...,0.783274,0.850343,0.006055,1217.391304,1323.913043,0.298085,1.087500,4.614122,5.017857,0.919540
3,0.35,3,5,2,62.2,59.0,4.51,4.48,2.79,2.595650,...,0.784708,0.850178,0.006209,1312.569522,1383.759733,0.289474,1.054237,4.485531,4.728814,0.948553
4,0.39,2,6,2,61.3,56.0,4.70,4.74,2.89,2.667353,...,0.783483,0.852760,0.006057,1186.440678,1298.728814,0.296837,1.094643,4.714519,5.160714,0.913540


In [22]:
cv = KFold(n_splits=5, shuffle=True, random_state=SEED)

# Set categorical features for catboost
cat_features = [col for col in X_train_prep.columns if X_train_prep[col].dtype == 'category']

regressors = {
    'LGBMRegressor1': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='gbdt'),
    # 'LGBMRegressor2': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='dart'),
    # 'LGBMRegressor3': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='goss'),
    # 'LGBMRegressor4': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='rf', subsample=.632, subsample_freq=1),
    # 'LGBMRegressor5': LGBMRegressor(random_state=SEED, n_jobs=-1, class_weight='balanced'),
    # 'LGBMRegressor6': LGBMRegressor(random_state=SEED, n_jobs=-1, subsample=0.7),
    # 'LGBMRegressor7': LGBMRegressor(random_state=SEED, n_jobs=-1, colsample_bytree=0.7),
    # 'LGBMRegressor8': LGBMRegressor(random_state=SEED, n_jobs=-1, subsample=0.7, colsample_bytree=0.7),
    # 'LGBMRegressor9': LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='dart', colsample_bytree=0.7),
    # 'XGBRegressor1': XGBRegressor(random_state=SEED, n_jobs=-1),
    # 'XGBRegressor2': XGBRegressor(random_state=SEED, n_jobs=-1, booster='dart'),
    # 'XGBRegressor3': XGBRegressor(random_state=SEED, n_jobs=-1, booster='gblinear'),
    # 'XGBRegressor4': XGBRegressor(random_state=SEED, n_jobs=-1, colsample_bytree=0.7),
    # 'XGBRegressor5': XGBRegressor(random_state=SEED, n_jobs=-1, subsample=0.7),
    # 'XGBRFRegressor6': XGBRegressor(random_state=SEED, n_jobs=-1, objective='reg:squarederror'),
    # 'XGBRandomForestRegressor': XGBRFRegressor(random_state=SEED, n_jobs=-1),
    # 'CatBoostRegressor': CatBoostRegressor(random_state=SEED, silent=True, cat_features=cat_features), # Promising but fails on the cv
    # 'RandomForestRegressor': RandomForestRegressor(random_state=SEED, n_jobs=-1),
    # 'ExtraTreesRegressor': ExtraTreesRegressor(random_state=SEED, n_jobs=-1),
    # 'AdaBoostRegressor': AdaBoostRegressor(random_state=SEED),
    # 'GradientBoostingRegressor': GradientBoostingRegressor(random_state=SEED),
    # 'BaggingRegressor': BaggingRegressor(random_state=SEED, n_jobs=-1),
    # 'KNeighborsRegressor': KNeighborsRegressor(n_jobs=-1),
    # 'DecisionTreeRegressor': DecisionTreeRegressor(random_state=SEED),
    # 'GaussianProcessRegressor': GaussianProcessRegressor(random_state=SEED),
    # 'MLPRegressor1': MLPRegressor(random_state=SEED, max_iter=1000, activation='relu', solver='adam'),
    # 'MLPRegressor2': MLPRegressor(random_state=SEED, max_iter=1000, activation='relu', solver='lbfgs'), # promising but long to train
    # 'MLPRegressor3': MLPRegressor(random_state=SEED, max_iter=5000, activation='tanh', solver='adam'),
    # 'MLPRegressor4': MLPRegressor(random_state=SEED, max_iter=1000, activation='tanh', solver='lbfgs'),  # promising but long to train
    # 'MLPRegressor5': MLPRegressor(random_state=SEED, max_iter=1000, activation='logistic', solver='adam'),
    # 'MLPRegressor6': MLPRegressor(random_state=SEED, max_iter=1000, activation='logistic', solver='lbfgs'),
    # 'MLPRegressor7': MLPRegressor(random_state=SEED, max_iter=1000, activation='identity', solver='adam'),
    # 'MLPRegressor8': MLPRegressor(random_state=SEED, max_iter=1000, activation='identity', solver='lbfgs'),
    # 'Ridge': Ridge(random_state=SEED),
    # 'SGDRegressor': SGDRegressor(random_state=SEED, max_iter=1000, tol=1e-3),
    # 'PassiveAggressiveRegressor': PassiveAggressiveRegressor(random_state=SEED, max_iter=1000, tol=1e-3),
    # 'Perceptron': Perceptron(random_state=SEED, max_iter=1000, tol=1e-3),
    # 'LinearRegression': LinearRegression(),
    # 'Lasso': Lasso(random_state=SEED),
    # 'ElasticNet': ElasticNet(random_state=SEED, max_iter=1e6),
    # 'HuberRegressor': HuberRegressor(max_iter=1000),
    # 'BayesianRidge': BayesianRidge(),
    # 'ARDRegression': ARDRegression(),
    # 'TheilSenRegressor': TheilSenRegressor(random_state=SEED),
    # 'RANSACRegressor': RANSACRegressor(random_state=SEED),
    # 'OrthogonalMatchingPursuit': OrthogonalMatchingPursuit(normalize=False),
    # 'Lars': Lars(),
    # 'LassoLars': LassoLars(),
    # 'LassoLarsIC': LassoLarsIC(normalize=False),
    # 'StackingRegressor': StackingRegressor(
    #         estimators=[
    #             ('LGBMRandomForestRegressor', LGBMRegressor(random_state=SEED, n_jobs=-1, boosting_type='rf', subsample=.632, subsample_freq=1)),
    #             ('XGBRandomForestRegressor', XGBRFRegressor(random_state=SEED, n_jobs=-1)),
    #             ('RandomForestRegressor', RandomForestRegressor(random_state=SEED, n_jobs=-1)),
    #             # ('ExtraTreesRegressor', ExtraTreesRegressor(random_state=SEED, n_jobs=-1))
    #             ], 
    #         final_estimator=Ridge(random_state=SEED),
    #         # cv=cv,
    #         # n_jobs=-1,
    #         )
}

for model_name, regressor in regressors.items():
    t0 = time.time()
    scores = []
    feature_importances = pd.DataFrame()
    ttr = TransformedTargetRegressor(regressor=regressor, func=target_transform, inverse_func=inverse_target_transform, check_inverse=False)
    
    for i, (train_index, test_index) in tqdm(enumerate(cv.split(X_train))):
        
        X_train_cv, X_test_cv = X_train.iloc[train_index].copy(), X_train.iloc[test_index].copy()
        y_train_cv, y_test_cv = y_train.iloc[train_index].copy(), y_train.iloc[test_index].copy()
        
        if data_prep_has_fit_method:
            X_train_cv, X_test_cv = data_prepation(X_train_cv, X_test_cv)
        
        ttr.fit(X_train_cv, y_train_cv)        
        y_pred = ttr.predict(X_test_cv)
        score_eval = mean_squared_error(y_test_cv, y_pred, squared=False)
        scores.append(score_eval)
        
        try:
            feature_importance = pd.Series(ttr.regressor_.feature_importances_, index=X_train_cv.columns, name=f'fold{i}')
        except:
            feature_importance = pd.Series(ttr.regressor_.coef_, index=X_train_cv.columns, name=f'fold{i}')
        feature_importances = pd.concat([feature_importances, feature_importance], axis=1)
    
    feature_importances['mean'] = feature_importances.mean(axis=1)
    
    ttr.fit(X_train_prep, y_train)
    y_pred = ttr.predict(X_test_prep)
    
    if not SUBMIT:
        score_eval = mean_squared_error(y_test, y_pred, squared=False)
    
    print(f'{model_name}: {np.mean(scores):.4f} ± {np.std(scores):.4f}, Time: {time.time() - t0:.2f} seconds, RMSE: {score_eval:.4f}')
    print(feature_importances.sort_values('mean', ascending=False))

5it [00:13,  2.80s/it]


LGBMRegressor1: 581.0963 ± 0.6164, Time: 16.96 seconds, RMSE: 571.6996
         fold0  fold1  fold2  fold3  fold4   mean
clarity    680    698    701    731    687  699.4
color      577    557    571    570    580  571.0
y          464    450    436    446    443  447.8
x          400    377    389    398    439  400.6
carat      339    358    350    338    309  338.8
z          172    182    179    168    172  174.6
depth      181    170    171    146    171  167.8
cut        106    120    116    116    123  116.2
table       81     88     87     87     76   83.8


In [39]:
regressor = LGBMRegressor(random_state=SEED, n_jobs=-1, colsample_bytree=.632, subsample=.632, subsample_freq=1)
ttr = TransformedTargetRegressor(regressor=regressor, func=target_transform, inverse_func=inverse_target_transform, check_inverse=False)

selected_cols = X_train.columns
len_cols = len(selected_cols)

# drop one feature at a time based on feature importance
for _ in range(len_cols):
    t0 = time.time()
    scores = []
    feature_importances = pd.DataFrame()
    for i, (train_index, test_index) in tqdm(enumerate(cv.split(X_train))):

        X_train_cv, X_test_cv = X_train.iloc[train_index][selected_cols].copy(), X_train.iloc[test_index][selected_cols].copy()
        y_train_cv, y_test_cv = y_train.iloc[train_index].copy(), y_train.iloc[test_index].copy()
        
        if data_prep_has_fit_method:
            X_train_cv, X_test_cv = data_prepation(X_train_cv, X_test_cv)
        
        ttr.fit(X_train_cv, y_train_cv)        
        y_pred = ttr.predict(X_test_cv)
        score_eval = mean_squared_error(y_test_cv, y_pred, squared=False)
        scores.append(score_eval)
        
        try:
            feature_importance = pd.Series(ttr.regressor_.feature_importances_, index=X_train_cv.columns, name=f'fold{i}')
        except:
            feature_importance = pd.Series(ttr.regressor_.coef_, index=X_train_cv.columns, name=f'fold{i}')
        feature_importances = pd.concat([feature_importances, feature_importance], axis=1)

    feature_importances['mean'] = feature_importances.mean(axis=1)

    ttr.fit(X_train_prep[selected_cols], y_train)
    y_pred = ttr.predict(X_test_prep[selected_cols])
    
    selected_cols = feature_importances.sort_values('mean', ascending=True).index[1:]

    if not SUBMIT:
        score_eval = mean_squared_error(y_test, y_pred, squared=False)

    print(f'{model_name}: {np.mean(scores):.4f} ± {np.std(scores):.4f}, Time: {time.time() - t0:.2f} seconds, RMSE: {score_eval:.4f}')
    print(feature_importances.sort_values('mean', ascending=False))
    # print(f'Selected columns: {selected_cols}')

5it [00:25,  5.19s/it]


LGBMRegressor1: 582.2221 ± 1.3039, Time: 38.59 seconds, RMSE: 574.5906
                      fold0  fold1  fold2  fold3  fold4   mean
clarity                 525    535    529    541    529  531.8
color                   425    422    425    419    405  419.2
carat                   242    237    246    242    228  239.0
y                       156    147    151    149    162  153.0
density                 134    133    132    136    132  133.4
z                       118    128    103    115    132  119.2
x                       126    107    114     97    133  115.4
volume_ratio10          131    113    102     98     99  108.6
cut                      89    100     98    102     94   96.6
surface_area             81     81     81     82     80   81.0
volume_ratio6            83     81     84     77     74   79.8
diagonal_distance        75     77     75     79     93   79.8
table_percentage         74     86     83     77     76   79.2
depth                    76     67     72     5

5it [00:17,  3.50s/it]


LGBMRegressor1: 582.7259 ± 1.6420, Time: 20.94 seconds, RMSE: 575.9152
                      fold0  fold1  fold2  fold3  fold4   mean
clarity                 496    500    495    509    500  500.0
color                   435    437    422    430    444  433.6
carat                   283    305    272    287    269  283.2
y                       141    137    128    148    136  138.0
volume_ratio14          132    123    103    118    130  121.2
x                       109    122    121    114    114  116.0
density                 124    109    121    103    100  111.4
z                       107    111    118    109    111  111.2
cut                      95     98     98     99     93   96.6
diagonal_distance        92     87     98     81    100   91.6
table_percentage         84     83     84     74     87   82.4
aspect_ratio_xy          75     80     83     82     89   81.8
girdle_diameter          72     71     67     58     69   67.4
depth_percentage         63     58     68     7

5it [00:16,  3.27s/it]


...

LGBMRegressor1: 919.1052 ± 9.9074, Time: 18.14 seconds, RMSE: 885.0564
         fold0  fold1  fold2  fold3  fold4    mean
carat     1842   1824   1856   1820   1826  1833.6
clarity   1158   1176   1144   1180   1174  1166.4
y            0      0      0      0      0     0.0


5it [00:04,  1.17it/s]


LGBMRegressor1: 4228.2854 ± 33.2994, Time: 5.17 seconds, RMSE: 4216.9971
         fold0  fold1  fold2  fold3  fold4   mean
clarity    700    700    700    700    700  700.0
carat        0      0      0      0      0    0.0


5it [00:03,  1.30it/s]


LGBMRegressor1: 4228.2854 ± 33.2994, Time: 4.69 seconds, RMSE: 4216.9971
         fold0  fold1  fold2  fold3  fold4   mean
clarity    700    700    700    700    700  700.0


In [ ]:
# y_pred_test = test_predictions.mean(axis=1).round().astype(int)

# sub = pd.read_csv('submissions/sample_submission.csv')
# sub['quality'] = y_pred_test
# now = time.strftime("%Y-%m-%d %H_%M_%S")
# sub.to_csv(f'submissions/submission{now}.csv', index=False)
# # Copy the leaked values from the original dataset before submitting
# # Transform the price column back to the original scale